In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
# from mlfs import config
# settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/zohra/mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

today = datetime.date.today()

location_str

2025-11-16 12:24:48,058 INFO: Initializing external client
2025-11-16 12:24:48,059 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 12:24:50,159 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279126


'[{"country": "spain", "city": "catalunya", "street": "barcelona-eixample", "aqicn_url": "https://api.waqi.info/feed/@6669/", "latitude": 41.85, "longitude": 1.57}, {"country": "spain", "city": "catalunya", "street": "barcelona-gracia-st.gervasi", "aqicn_url": "https://api.waqi.info/feed/@11758", "latitude": 41.85, "longitude": 1.57}, {"country": "spain", "city": "catalunya", "street": "barcelona-palau-reial", "aqicn_url": "https://api.waqi.info/feed/@10532", "latitude": 41.85, "longitude": 1.57}, {"country": "spain", "city": "catalunya", "street": "barcelona-parc-vall-dhebron", "aqicn_url": "https://api.waqi.info/feed/@10533/", "latitude": 41.85, "longitude": 1.57}, {"country": "spain", "city": "catalunya", "street": "barcelona-poblenou", "aqicn_url": "https://api.waqi.info/feed/@10534/", "latitude": 41.85, "longitude": 1.57}, {"country": "spain", "city": "catalunya", "street": "barcelona", "aqicn_url": "https://api.waqi.info/feed/@11762", "latitude": 41.85, "longitude": 1.57}]'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_per_city',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_spain',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd
aq_todays=[]
for loc in location:
    country  = loc["country"]
    city     = loc["city"]
    street   = loc["street"]
    aqicn_url = loc["aqicn_url"]
    latitude = loc["latitude"]
    longitude = loc["longitude"]
    aq_today = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    aq_todays.append(aq_today)
aq_today_df = pd.concat(aq_todays, ignore_index=True)

In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     6 non-null      float32       
 1   country  6 non-null      object        
 2   city     6 non-null      object        
 3   street   6 non-null      object        
 4   date     6 non-null      datetime64[ns]
 5   url      6 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 396.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 42.0°N 1.5°E
Elevation 814.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-16,13.50,0.0,17.373497,264.053223,catalunya
1,2025-11-17,12.65,0.0,8.766573,199.179108,catalunya
2,2025-11-18,6.80,0.0,6.489992,236.309906,catalunya
3,2025-11-19,5.15,0.0,6.287130,246.370605,catalunya
4,2025-11-20,4.85,0.0,7.895416,223.152405,catalunya
5,2025-11-21,4.90,0.0,23.411074,358.237640,catalunya
6,2025-11-22,7.05,0.0,7.172949,252.474335,catalunya


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-16 12:25:01,051 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279126/fs/1265733/fg/1703457


Uploading Dataframe: 100.00% |█| Rows 6/6 | Elapsed Time: 00:02 | Remaining Time


Launching job: air_quality_per_city_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279126/jobs/named/air_quality_per_city_1_offline_fg_materialization/executions


(Job('air_quality_per_city_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747697
         }
       },
       "result": {
         "observed_value": 17.0,
         "element_count": 6,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-16T11:25:01.000051Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful

In [10]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-16 12:25:17,031 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279126/fs/1265733/fg/1721705


Uploading Dataframe: 100.00% |█| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time


Launching job: weather_spain_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279126/jobs/named/weather_spain_1_offline_fg_materialization/executions
2025-11-16 12:25:35,397 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-16 12:25:38,631 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-16 12:27:24,351 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-16 12:27:24,529 INFO: Waiting for log aggregation to finish.
2025-11-16 12:27:57,163 INFO: Execution finished successfully.


(Job('weather_spain_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761872
         }
       },
       "result": {
         "observed_value": 6.287129878997803,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-16T11:25:17.000030Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_t

In [11]:
air_quality_fg_v2 = fs.get_feature_group(
    name='air_quality_per_city',
    version=2,
)
air_quality_df_v2=air_quality_fg_v2.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.86s) 


In [12]:
air_quality_df_v2.head()

,date,pm25,street,url,country,city,rolling_mean
0,2025-09-13 00:00:00+00:00,40.0,barcelona-gracia-st.gervasi,https://api.waqi.info/feed/@11758,spain,catalunya,35.333332
1,2025-06-05 00:00:00+00:00,43.0,barcelona-poblenou,https://api.waqi.info/feed/@10534/,spain,catalunya,33.000000
2,2023-08-02 00:00:00+00:00,49.0,barcelona-gracia-st.gervasi,https://api.waqi.info/feed/@11758,spain,catalunya,31.333334
3,2023-04-01 00:00:00+00:00,50.0,barcelona-gracia-st.gervasi,https://api.waqi.info/feed/@11758,spain,catalunya,25.000000
4,2025-06-09 00:00:00+00:00,55.0,barcelona-gracia-st.gervasi,https://api.waqi.info/feed/@11758,spain,catalunya,60.333332


In [13]:
air_quality_df_v2=air_quality_df_v2.sort_values("date", ascending=False)
#air_quality_df_v2.head()
new_rolling_mean=air_quality_df_v2.iloc[0:3]["pm25"].mean()

In [14]:
aq_today_df_v2 = aq_today_df.copy()
aq_today_df_v2["rolling_mean"] = new_rolling_mean
aq_today_df_v2

,pm25,country,city,street,date,url,rolling_mean
0,65.0,spain,catalunya,barcelona-eixample,2025-11-16,https://api.waqi.info/feed/@6669/,56.0
1,65.0,spain,catalunya,barcelona-gracia-st.gervasi,2025-11-16,https://api.waqi.info/feed/@11758,56.0
2,17.0,spain,catalunya,barcelona-palau-reial,2025-11-16,https://api.waqi.info/feed/@10532,56.0
3,17.0,spain,catalunya,barcelona-parc-vall-dhebron,2025-11-16,https://api.waqi.info/feed/@10533/,56.0
4,17.0,spain,catalunya,barcelona-poblenou,2025-11-16,https://api.waqi.info/feed/@10534/,56.0
5,17.0,spain,catalunya,barcelona,2025-11-16,https://api.waqi.info/feed/@11762,56.0


In [15]:
aq_today_df_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   pm25          6 non-null      float32       
 1   country       6 non-null      object        
 2   city          6 non-null      object        
 3   street        6 non-null      object        
 4   date          6 non-null      datetime64[ns]
 5   url           6 non-null      object        
 6   rolling_mean  6 non-null      float32       
dtypes: datetime64[ns](1), float32(2), object(4)
memory usage: 420.0+ bytes


In [16]:
air_quality_fg_v2.insert(aq_today_df_v2)

2025-11-16 12:28:02,130 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279126/fs/1265733/fg/1721706


Uploading Dataframe: 100.00% |█| Rows 6/6 | Elapsed Time: 00:01 | Remaining Time


Launching job: air_quality_per_city_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279126/jobs/named/air_quality_per_city_2_offline_fg_materialization/executions


(Job('air_quality_per_city_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761874
         }
       },
       "result": {
         "observed_value": 17.0,
         "element_count": 6,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-16T11:28:02.000129Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
